<a href="https://colab.research.google.com/github/casualcomputer/redis-rq-llm/blob/master/google_colab_rq_llm_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Large-Scale Querying of LLM Endpoints (CPU only) Using Redis Queue

We are testing the feasibility of querying a large language model (LLM) endpoint at scale using Redis Queue (RQ). This approach allows us to handle and process a high volume of queries efficiently by distributing them across multiple workers for parallel processing.

Our goal is to ensure the system can manage substantial simultaneous requests, maintain stability, and provide timely responses. By leveraging Redis Queue, we hope to optimize resource utilization, enhance scalability, and improve fault tolerance.

This test will help identify bottlenecks, determine practical limits, and guide necessary improvements to achieve efficient and reliable large-scale query handling. However, we noted during the experiment that results are returned in sequence, despite expecting certain queries to be processed simultaneously.

## Download folders and install packages

In [ ]:
! pip install redis rq requests fastapi llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

## Update linux packages and install redis-server

In [ ]:
!sudo apt-get update
!sudo apt-get install redis-server

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,189 kB]
Get:13 http://archive.ubuntu.com/ubuntu j

In [ ]:
!sudo service redis-server start

Starting redis-server: redis-server.


## Donwload model from huggingface

In [ ]:
from huggingface_hub import hf_hub_download
from google.colab import userdata

model_name = "cjpais/llava-1.6-mistral-7b-gguf"
model_file = "llava-v1.6-mistral-7b.Q4_K_M.gguf"

# save your huggingface access key as HF_TOKEN in the colab secret before you continue
model_path = hf_hub_download(model_name, filename=model_file, local_dir='/content/redis-rq-llm/models/', token=userdata.get('HF_TOKEN'))
print("Model path:", model_path)

llava-v1.6-mistral-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Model path: /content/redis-rq-llm/models/llava-v1.6-mistral-7b.Q4_K_M.gguf


## Create an API endpoint for the LLM

In [ ]:
%%writefile fastapi_app.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from llama_cpp import Llama

# Initialize the FastAPI app
app = FastAPI()

# Load the LLM model
model_path = "/content/redis-rq-llm/models/llava-v1.6-mistral-7b.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

# Define request and response models
class QueryRequest(BaseModel):
    question: str

class QueryResponse(BaseModel):
    answer: str

# Define the query endpoint
@app.post("/query", response_model=QueryResponse)
async def query_llm(request: QueryRequest):
    system_message = "You are a helpful assistant"
    user_message = f"Q: {request.question} A: "

    prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

    try:
        # Run the model to get the response
        output = llm(
            prompt,  # Prompt
            max_tokens=2000,  # Generate up to 2000 tokens
            stop=["Q:", "\n"],  # Stop generating just before the model would generate a new question
            echo=False  # Do not echo the prompt back in the output
        )

        # Extract and return the response
        response_text = output["choices"][0]["text"].strip()

        # Ensure the response is trimmed properly
        response_text = response_text.split("[/INST]")[-1].strip()

        return QueryResponse(answer=response_text)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Run the FastAPI application
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Template for sending a request
# curl -X POST "http://localhost:8000/query" -H "Content-Type: application/json" -d "{\"question\": \"Name the planets in the solar system?\"}"

Writing fastapi_app.py


## Quietly serve the LLM API in the background

In [ ]:
import subprocess

# Start the FastAPI server in the background
fastapi_process = subprocess.Popen(['python', 'fastapi_app.py'])

# You can also add a brief sleep to ensure the server starts before continuing
import time
time.sleep(5)  # Sleep for 5 seconds to give the server time to start

Testing the fastapi_app.py (LLM endpoint)

In [ ]:
!curl -X POST "http://localhost:8000/query" -H "Content-Type: application/json" -d "{\"question\": \"Name the planets in the solar system?\"}"

{"answer":"The planets in our solar system, from closest to the sun, are:"}

## Write a function to query LLM API and log progress

In [ ]:
%%writefile tasks.py
import requests
import time

def process_question(question, url="http://localhost:8000/query"):
    """
    Send a question to the FastAPI server and log the response time.

    Args:
        question (str): The question to send.
        url (str): The API endpoint to send the question to.

    Returns:
        dict: A dictionary containing the question, response, and timing information.
    """
    # Record the start time
    start_time = time.time()

    # Send the POST request to the FastAPI server
    response = requests.post(url, json={"question": question})

    # Record the end time
    end_time = time.time()

    # Calculate the duration taken to get the response
    duration = end_time - start_time

    # Create a result dictionary
    result = {
        "question": question,
        "response": response.text,
        "start_time": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
        "end_time": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
        "duration": duration
    }

    # Print the result (or save it to a log, etc.)
    print(result)
    print("-" * 60)  # Print a separator line for clarity

    return result


Writing tasks.py


## Implement a Redis Queue

In [ ]:
%%writefile main.py
from rq import Queue
from redis import Redis
from tasks import process_question

# Redis connection
redis_conn = Redis()

# RQ Queue
queue = Queue(connection=redis_conn)

# List of questions
questions = [
    "Is the mind the same as the brain, or do we have souls?",
    "Can computers think, or fall in love?",
    "Can computers be creative?",
    "What is consciousness?",
    "Can we really know what it feels like to be a bat?",
    "When you have a toothache, is the pain in your mouth or in your brain?",
    "What is an emotion?",
    "Is love just a feeling?",
    "How is love different from passion or sexual desire?",
    "Are emotions irrational?"]

# Enqueue each question to be processed
for index, question in enumerate(questions, start=1):
    job = queue.enqueue(process_question, question)
    print(f"Enqueued job {index}: {job.id}")

Writing main.py


In [ ]:
!redis-server

4451:C 25 Jun 2024 00:17:43.698 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
4451:C 25 Jun 2024 00:17:43.698 # Redis version=6.0.16, bits=64, commit=00000000, modified=0, pid=4451, just started
4451:C 25 Jun 2024 00:17:43.698 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
4451:M 25 Jun 2024 00:17:43.699 # Could not create server TCP listening socket *:6379: bind: Address already in use


In [ ]:
!python main.py
!rq worker

## The dollowing didn't work...
# # Start multiple RQ workers
# num_workers = 4
# worker_processes = []
# for _ in range(num_workers):
#     worker_process = subprocess.Popen(['rq', 'worker'])
#     worker_processes.append(worker_process)

# # Run the main script to enqueue jobs
# main_process = subprocess.Popen(['python', 'main.py'])

Enqueued job 1: 4063ef3f-64d0-4a90-a230-e039cec789bc
Enqueued job 2: 19aa5949-cd9b-474e-9f91-3700e0aeacfc
Enqueued job 3: 07c1db2f-1734-4668-832e-9243a09b2eed
Enqueued job 4: 5893c987-7696-4477-9731-e57eb6ec9975
Enqueued job 5: df30ac83-8eeb-449e-be62-11c105e143b1
Enqueued job 6: 0f34b969-b035-4a32-aeef-870f9bcd7a9b
Enqueued job 7: f7951a49-3976-46eb-aec3-bafd2ec61615
Enqueued job 8: 3466c458-8584-4be9-809c-7501d489d5f6
Enqueued job 9: 3ea12c4c-b28c-4f13-a6f3-bfa0506e4bd2
Enqueued job 10: b241d906-97e8-4c87-b667-0328182823d7
00:26:41 Worker rq:worker:b2e8bd860a784ba8ad1e5cc29a17bd66 started with PID 6647, version 1.16.2
00:26:41 Subscribing to channel rq:pubsub:b2e8bd860a784ba8ad1e5cc29a17bd66
00:26:41 *** Listening on default...
00:26:41 Cleaning registries for queue: default
00:26:41 default: tasks.process_question('Are emotions irrational?') (e1454d29-8297-4cb5-9110-ebbea38699d3)
{'question': 'Are emotions irrational?', 'response': '{"answer":"Emotions can be both rational and irrat